In [ ]:
#------------ dblp title match -------------#
# -*- coding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import pandas as pd
import json

results_dblp_title_match = {'result_CHI':[], 'result_UBI':[], 'result_CSCW':[], 'result_UIST':[]}
conf_name_map = {1163450153:'CHI', 1195049314:'CSCW', 1171345118:'UBI', 1166315290:'UIST'}
paper = []
for line in open('dblp_papers_with_s2_abstracts.jsonlist', 'r'):
    paper.append(json.loads(line))
df = pd.DataFrame(paper)

data_papers_HCI = {'data_CHI':[], 'data_UBI', 'data_CSCW':[], 'data_UIST':[]}
for conf_id, conf_name in conf_name_map.items():
    data_papers_HCI['data_{}'.format(conf_name)] = df[(df['dblp_venue'] == conf_name)][['dblp_title', 'dblp_venue', 's2_abstract']]
    data_papers_HCI['data_{}'.format(conf_name)]['papertitle'] = [title.encode('utf-8').strip().lower() for title in data_papers_HCI['data_{}'.format(conf_name)]['dblp_title']]
    data_papers_HCI['data_{}'.format(conf_name)]['papertitle'].to_csv('dblp_{}_title.tsv'.format(conf_name))
    data_papers_HCI['data_{}'.format(conf_name)]['s2_abstract'].to_csv('dblp_{}_abstract.tsv'.format(conf_name))

In [ ]:
# -*- coding: utf-8 -*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import pandas as pd

for conf_name in ['CHI', 'CSCW', 'UBI', 'UIST']:
    hci_tilte = pd.read_csv("../data/papertitle_{}.tsv".format(conf_name), sep = ",")

    hci_tilte['papertitle'] = [title.encode('utf-8').strip().lower() for title in hci_tilte['title']]

    from fuzzywuzzy import process

    # matched = 'Technology heirlooms?: considerations fo passing down and inheriting digital materials.'
    # print(papers['papertitle'].iloc[0])
    # highest = process.extractOne(str(papers['papertitle'].iloc[0]), hci_title['papertitle'])
    # print(highest)

    result = []
    with open('dblp_{}_title.tsv'.format(conf_name), "r") as title_f:
        for line in title_f:
            paper_idx = line[0:line.index(',')]
            paper_title = line[line.index(',') + 1:]
            highest = process.extractOne(paper_title, hci_tilte['papertitle'])
            print(highest)
            try:
                output = [highest[0], highest[1], highest[2], paper_idx]
                result.append(list(output))
            except:
                print(">>> error")
        result_pd = pd.DataFrame(data=result, columns=['papertitle', 'score', 'paperidx', 'dblppaperidx'])
        result_pd.to_csv('matched_{}.tsv'.format(conf_name))
